In [6]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import sys
sys.path.append("../../llm_engineering")
from api_clients import create_clients

In [7]:
clients = create_clients()

In [25]:
context = {} 
employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
  name = employee.split(' ')[-1][:-3]
  print("NAME: ", name)
  doc = ""
  with open(employee, "r", encoding="utf-8") as f:
    doc = f.read()
  context[name] = doc


NAME:  Chen
NAME:  Harper
NAME:  Thomson
NAME:  Lancaster
NAME:  Carter
NAME:  Tran
NAME:  Blake
NAME:  Bishop
NAME:  Thompson
NAME:  Spencer
NAME:  Greene
NAME:  Trenton


In [26]:
products = glob.glob("knowledge-base/products/*")

for product in products:
  name = product.split(os.sep)[-1][:-3]
  doc = ""
  with open(product, "r", encoding="utf-8") as f:
    doc = f.read()
  context[name] = doc

In [30]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [41]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context     

In [42]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [52]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] 
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    stream = clients["groq"].chat.completions.create(model=clients["models"]["GROQ_MODEL"], messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [53]:
view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
